In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!curl -L 'https://drive.google.com/uc?export=download&id=1uUETGOKiSrbdUzjjk758avUQeovdjwIc&confirm=t' > DeLT_distilbert-classification.zip
!unzip DeLT_distilbert-classification.zip -d DeLT_distilbert-classification

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  766M  100  766M    0     0  72.3M      0  0:00:10  0:00:10 --:--:-- 52.1M
Archive:  DeLT_distilbert-classification.zip
 extracting: DeLT_distilbert-classification/config.json  
 extracting: DeLT_distilbert-classification/optimizer.pt  
 extracting: DeLT_distilbert-classification/pytorch_model.bin  
 extracting: DeLT_distilbert-classification/rng_state.pth  
 extracting: DeLT_distilbert-classification/scheduler.pt  
 extracting: DeLT_distilbert-classification/trainer_state.json  
 extracting: DeLT_distilbert-classification/training_args.bin  


In [ ]:
#load temporary model
!curl -L 'https://drive.google.com/uc?export=download&id=1X-ty6_UcAxWFHlO9dDbKYz5H50qRnZpH&confirm=t' > DeiT_CNN.pt
!curl -L 'https://drive.google.com/uc?export=download&id=1h5Jvvbjy2mK3ncoeiuTM57pHefgjCS20&confirm=t' > TIMESERIES_LSTM.h5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  159M  100  159M    0     0  35.0M      0  0:00:04  0:00:04 --:--:-- 43.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
100 9307k  100 9307k    0     0  2157k      0  0:00:04  0:00:04 --:--:-- 8023k


#DistilBert - Span Extraction

In [ ]:
#LOGGER
import sys
from termcolor import colored
# Basic logging infrastructure. Simpler than the python 'logging' module.
def info(message):
    print(colored(message, 'blue'), file=sys.stderr)
def warn(message):
    print(colored(message, 'yellow'), file=sys.stderr)
def error(message):
    print(colored(message, 'red'), file=sys.stderr)

In [ ]:
! pip install yt_dlp webvtt-py transformers datasets ffmpeg timm > /dev/null 2>&1

In [ ]:
import os
import threading
import pandas as pd
import numpy as np
import time
from concurrent.futures import thread
from glob import glob
import sys
import multiprocessing.dummy
import subprocess
from sys import stdout
import yt_dlp
from yt_dlp import DownloadError, SameFileError
import time,re
import re
from time import time
from typing import Iterable, Iterator, List, Optional, Tuple
from enum import Enum
from datetime import timedelta
import random
import webvtt
from typing import Generator, Iterable, List, Optional, Tuple
from numpy import isin
import torch
import transformers
from datasets import Dataset, IterableDataset, IterableDatasetDict, ClassLabel, load_dataset, load_from_disk, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import pyarrow as pa
import itertools
from collections import defaultdict
from numpy.lib.function_base import delete
from termcolor import colored
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, precision_recall_curve, roc_curve
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Bidirectional
from keras.layers import Dropout
import requests,json
import keras
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import copy
%matplotlib inline
import torchvision
from torchvision import datasets
from torchvision import transforms as T # for simplifying the transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models
from PIL import Image, ImageFile
import timm
from timm.loss import LabelSmoothingCrossEntropy # This is better than normal nn.CrossEntropyLoss
import PIL
from PIL import Image

In [ ]:
#Global vars to store final predicted range given by the respective models
DistilBert, LSTM, DeiT = (0,0), (0,0), (0,0)

In [ ]:
list1=[]
class Caption:
	def __init__(self, start: int, end: int, text: str):
		self.start = start
		self.end = end
		self.text = text
		# self.is_sponsor = False

	def __repr__(self):
		text = self.text.replace('\n', '\\n')   #we remove the esc seq and concat the two lines.
		return f'Caption{{{self.start}:{self.end},{text}}}'

class DBSegment:
	category: str
	startTime: float
	endTime: float
	UUID: str
	userID: str
	votes: int
	views: int
	locked: int
	hidden: int
	shadowHidden: int
	videoID: str
	videoDuration: int
	reputation: int
	hashedVideoID: str
	timeSubmitted: int
	userAgent: str
	service: str
	description: str

class OverlappingSegmentGroup:
	def __init__(self):
		self.segments: List[DBSegment] = []
		self.votes = 0

def get_caption_list_from_path(path: str):
	cap_iter: Iterator[webvtt.Caption] = webvtt.read(path)  #loads the .vtt file from the path
	# The cap_iter may contain overlapping caption regions like below:
	#	<Caption start=00:00:00.060 end=00:00:01.730 text= \nyou know working with tech all the time>
	#	<Caption start=00:00:01.730 end=00:00:01.740 text=you know working with tech all the time\n >
	#	<Caption start=00:00:01.740 end=00:00:03.470 text=you know working with tech all the time\nI sometimes forget that not everyone>
	#	<Caption start=00:00:03.470 end=00:00:03.480 text=I sometimes forget that not everyone\n >
	#	<Caption start=00:00:03.480 end=00:00:05.059 text=I sometimes forget that not everyone\nknows about even the most basic of>
	#	<Caption start=00:00:05.059 end=00:00:05.069 text=knows about even the most basic of\n >
	#	<Caption start=00:00:05.069 end=00:00:07.579 text=knows about even the most basic of\nthings example well the various>
	#	<Caption start=00:00:07.579 end=00:00:07.589 text=things example well the various\n >
	#	<Caption start=00:00:07.589 end=00:00:10.070 text=things example well the various\nstandards of network cables cat 5 versus>
	#	<Caption start=00:00:10.070 end=00:00:10.080 text=standards of network cables cat 5 versus\n >
	# This is how auto-generated captions follow a person's speech.
	# cap[1] replaces cap[0] on-screen and creates a sliding effect.
	# This is fine for YT but not for us.

	# To fix remove the duplication, we check for overlapping parts of text
	# between the captions and eliminate them.
	output: List[Caption] = []
	prev_cap = Caption(start=0, end=0, text="")
	for cap in cap_iter:
		cap = Caption(cap.start_in_seconds, cap.end_in_seconds, cap.text)
		cap.text = clean_text(cap.text)  #clean_text() returns plain text with no punc,

		if cap.text == '':
			continue

		ilen = get_intersection_length(prev_cap.text, cap.text)
		# Is the overlap a whole token or more?
		if ilen > 0 and ilen >= len(cap.text.split(' ', 1)[0]):
			if len(cap.text) == ilen:
				# Remove the whole caption altogether, it is duplicated
				continue
			else:
				# Remove the overlap from this caption
				cap.text = cap.text[ilen:]

		output.append(cap)
		prev_cap = cap

	return output

def get_intersection_length(left: str, right: str):
	"""
	Finds how many characters of overlap is there between left and right.

	```
	left  = "except the various"
	right =            "various forms"
						^^^^^^^
	```
	"""
	i = 0
	while not right.startswith(left[i:]):
		i += 1
	ilen = len(left) - i
	assert ilen == 0 or left[-ilen:] == right[:ilen]
	return ilen

def clean_text(text: str) -> str:
	# Clean italicised quotes
	text = text.replace('\u2019', '\'')
	# Clean HTML and ASCII whitespace
	text = re.sub(r"&nbsp;|\s+", " ", text, flags=re.MULTILINE)
	# Remove markers like [Music], (Intro), special ... and emphasis (*) characters
	text = re.sub(r"\[.+?\]|\(.+?\)|\u2026\*", "", text)
	return text.strip()

def build_segment_groups(segments: List[DBSegment]) -> List[OverlappingSegmentGroup]:
	"""
	This function will find segments that are contained inside of eachother, called similar segments.  (Timestamps)
	Segments with less than -1 votes are already ignored before this function is called.

	Based on https://github.com/ajayyy/SponsorBlockServer/blob/e74b985304443b17b429c5c82696c7a03e78a166/src/routes/getSkipSegments.ts#L276
	"""

	# Create groups of segments that are similar to eachother
	# Segments must be sorted by their startTime so that we can build groups chronologically:
	# 1. As long as the segments' startTime fall inside the currentGroup, we keep adding them to that group
	# 2. If a segment starts after the end of the currentGroup (> cursor), no other segment will ever fall
	#    inside that group (because they're sorted) so we can create a new one
	overlappingSegmentsGroups: List[OverlappingSegmentGroup] = []
	currentGroup = None
	cursor = -1 # -1 to make sure that, even if the 1st segment starts at 0, a new group is created
	for segment in segments:
		if segment.startTime >= cursor:
			currentGroup = OverlappingSegmentGroup()
			overlappingSegmentsGroups.append(currentGroup)

		currentGroup.segments.append(segment)
		# only if it is a positive vote, otherwise it is probably just a sponsor time with slightly wrong time
		if segment.votes > 0:
			currentGroup.votes += segment.votes

		cursor = max(cursor, segment.endTime)

	return overlappingSegmentsGroups

def get_best_segment(group: OverlappingSegmentGroup):
	"""
	SponsorBlock chooses a segment from an overlap group randomly by using the
	votes property as a weight. This is done so that all segments can have a
	chance of appearing and makes sense in that system, but here we just
	want the best possible match, hence we pick the segment with the highest vote.
	"""

	return max(group.segments, key=lambda segment: segment.votes)

def prepare_data(captions_path: str, output_path: str, vote_threshold: int, chunk_size: int):
	filenames = os.listdir(captions_path)
	rows = []

	start_time = time()

	chunk_id = 0
	def write_chunk():
		nonlocal chunk_id
		nonlocal rows
		chunk_id += 1
		df = pd.DataFrame(rows, columns=['video_id', 'captions', 'sponsor_times'])  # we are initiating pandas df with 3 cols and storing corresponding data in it.
		filename, ext = output_path.rsplit('.json', 1)
		chunk_filename = f'{filename}.{chunk_id}.json{ext}'                                    #writing pandas df to json with chunksize 20,000 per json file.
		df.to_json(chunk_filename, compression='infer', orient='records', lines=True)
		rows = []

	print('Processing captions...')
	for i, filename in enumerate(filenames):                #filenames is basically .vtt caption file path
		progress = (i + 1) / len(filenames) * 100
		elapsed = time() - start_time
		remaining = (100 - progress) * elapsed / progress
		print('\u001b[2K\r', end='')
		print(f'{progress:.2f}% ' +
			f'elapsed: {timedelta(seconds=int(elapsed))}, ' +
			f'remaining: {timedelta(seconds=int(remaining))}',
			end='', flush=True)
		videoID = filename.split('.')[0]         #filenames= [videoid1videoid2]

		try:
			captions = get_caption_list_from_path(f'{captions_path}/{filename}')
		except Exception as e:
			e.args = (*e.args, f'while processing {filename}')
			raise e

		captions_as_tuples = [(cap.text, cap.start, cap.end) for cap in captions]
		rows.append((videoID, captions_as_tuples,list1))                              #calling the above explained funcs and storing captions & sponsor segment with start,end times
                                                                                           # {"video_id":"--ABCD","captions":[["Hey....",0.16,6.24],[],[] ],"sponsor_times":[[101.29842,128.87369]]}
		if len(rows) > chunk_size:
			write_chunk()

	if len(rows) > 0:
		write_chunk()

def get_or_default(arr, i, default):
	return arr[i] if i < len(arr) else default

CAPTIONS_PATH = '/content/TestFolder1'#'/content/sponsor'
OUTPUT_PATH =  '/content/TestFolder2/data.json.gz'#'/content/myfiles/data.json.gz'
VOTE_THRESHOLD = 1 # Higher threshold than minimum (-1) for SponsorBlock
CHUNK_SIZE = 1
def main():
	!mkdir -p /content/TestFolder2
	prepare_data(CAPTIONS_PATH, OUTPUT_PATH, VOTE_THRESHOLD, CHUNK_SIZE)

#if __name__ == '__main__':
#	main()

In [ ]:
#SEQUENCE CLASSIFICATION
sys.path.append(os.path.dirname(os.path.realpath('..')))
#from data_loader import load_examples_from_chunks, load_captions_from_chunks

os.environ["WANDB_DISABLED"] = "true"

In [ ]:
#Tokenize the datatset with the pre-trained tokenizer. Sequences are padded to the maximum length supported by BERT and truncated if longer.
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
model = None
trainer = None
trained = None

def softmax_outputs(outputs) -> dict:
    return torch.nn.functional.softmax(outputs.logits, dim=-1)[0].tolist()

trained = AutoModelForSequenceClassification.from_pretrained('/content/DeLT_distilbert-classification')   #7210   7449 drive/MyDrive/distilbert-classification-uncased/


trained.to('cuda') #Uncomment this if you are using Nvidia GPU

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
#from data_loader import Caption, load_captions_from_chunks, segment_text, get_intersection_range

def caption_times(c):
    return c.start, c.end

def prediction_times(p):
    return tuple(p[0])

def tumbling_time_window(captions, duration, key=caption_times):
    results = [captions[0]]
    for caption in captions:
        if key(results[-1])[1] - key(results[0])[0] <= duration:
            results.append(caption)
        else:
            yield results
            results = [caption]

    yield results

def session_time_window(captions, duration, key=caption_times):

    captions_iter = iter(captions)
    results = [next(captions_iter)]
    for caption in captions_iter:
        if key(results[-1])[1] - key(caption)[0] <= duration:
            results.append(caption)
        else:
            yield results
            results = [caption]

    yield results

def batch(iterable, n):
    length = len(iterable)
    for i in range(0, length, n):
        yield iterable[i:min(i + n, length)]

def decode_label(outputs):
    content, sponsor = outputs

    prediction_dict = {'sponsor': sponsor, 'content': content}
    prediction_dict = {k: v for k, v in sorted(prediction_dict.items(), key=lambda item: item[1], reverse=True)}

    return next(iter(prediction_dict.items()))

def predict_in_batches(texts, batch_size: int = 8):
    batches = list(batch(texts, batch_size))
    for b in batches:
        inputs = defaultdict(list)
        for text in b:
            tokenized = tokenize_function({ 'text': text })
            for k, v in tokenized.items():
                inputs[k].append(v)

        inputs = { k: torch.tensor(v).cuda() for k, v in inputs.items() }               # IF MODEL WAS LOADED TO GPU THEN ADD ".cuda()" to torch.tensor(v)
        outputs = trained(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1).tolist()
        yield from predictions

def predict_sponsor_segments(captions, window_duration=10):
    windows = list(tumbling_time_window(captions, window_duration))
    window_texts = [segment_text(window) for window in windows]
    predictions = predict_in_batches(window_texts, 4)

    for window, text, prediction in zip(windows, window_texts, predictions):
        yield [window[0].start, window[-1].end], text, *decode_label(prediction)

def merge_prediction_(predictions):
    assert len(set((label for _, _, label, _ in predictions))) == 1
    # All co-occurring predictions have the same label so we merge them
    merged_start, merged_end = predictions[0][0][0], predictions[-1][0][1]
    merged_text = ' '.join((text for _, text, _, _ in predictions))
    # Don't know what the correct way to compute the joint probability here is,
    # just assume they are independent; We don't really use this number anywhere
    prob = np.prod([prob for _, _, _, prob in predictions])
    return [merged_start, merged_end], merged_text, predictions[0][2], prob

def merge_predictions(predictions, within_duration=5):
    for co_occuring in session_time_window(predictions, within_duration, key=prediction_times):
        merged = [co_occuring[0]]
        for times, text, label, prob in co_occuring[1:]:
            _, _, prev_label, _ = merged[0]
            if label == prev_label:
                merged.append((times, text, label, prob))
            else:
                yield merge_prediction_(merged)
                merged = [(times, text, label, prob)]

        if len(merged) > 0:
            yield merge_prediction_(merged)

In [ ]:
def range_equals(left: 'Tuple[float, float]', right: 'Tuple[float, float]', eps: float) -> bool:
    left_start, left_end = left
    right_start, right_end = right

    return (abs(left_start - right_start) <= eps
        and abs(left_end - right_end) <= eps)

def count_range_equals(pairs, eps: float) -> int:
    cnt = 0
    for left, right in pairs:
        if range_equals(left, right, eps):
            cnt += 1
    return cnt

assert range_equals([0, 5], [0, 5], eps=0)
assert range_equals([1, 6], [0, 5], eps=1)
assert range_equals([-1, 4], [0, 5], eps=1)
assert not range_equals([-2, 4], [0, 5], eps=1)
assert not range_equals([1, 7], [0, 5], eps=1)

def range_negation(base: 'Tuple[float, float]', ranges: 'List[Tuple[float, float]]') -> 'List[Tuple[float, float]]':
    """
    base:    |-------------|
    ranges:  | ***   **    |
    Return:  |#   ###  ####|
    """
    results = []
    last_end = base[0]
    for r in ranges:
        if last_end != r[0]:
            results.append((last_end, r[0]))
        last_end = r[1]
    if last_end != base[1]:
        results.append((last_end, base[1]))

    return results

assert range_negation((2, 10), [(3,4), (5, 6)]) == [(2, 3), (4, 5), (6, 10)]
assert range_negation((2, 6), [(3,4), (5, 6)]) == [(2, 3), (4, 5)]
assert range_negation((3, 6), [(3,4), (5, 6)]) == [(4, 5)]

In [ ]:
def create_labels_from_range(captions, sponsor_ranges):
    token_labels = []
    try:
      caption_labels = np.zeros(len(captions), dtype=bool)
      for start_idx, end_idx in sponsor_ranges:
          if start_idx is None or end_idx is None:
              continue
          for i in range(start_idx, end_idx + 1):
              caption_labels[i] = True

      token_labels = []
      for i, caption in enumerate(captions):
          num_tokens = len(caption.text.split())
          token_labels.extend([caption_labels[i]] * num_tokens)
    except:
       token_labels = []
    return token_labels

def create_labels_from_times(captions, sponsor_times):
    try:
      ranges = [get_intersection_range(captions, *pair[1]) for pair in sponsor_times]
    except:
      ranges = [(0,0)]

    return create_labels_from_range(captions, ranges)

def evaluate(videos, eps=5, window_duration=10):
    global DistilBert
    predicted_labels = np.empty(0)
    actual_labels = np.empty(0)
    # Values for our close match metric (exact match with threshold)
    # Number of maches
    close_matches = 0
    # Number of predicted ranges
    total_predicted_ranges = 0

    for video_id, captions, sponsor_ranges in tqdm(videos):
        print(colored(f'Input Video ID: {video_id}', None, 'on_magenta'))
        sponsor_times = [(captions[start].start, captions[end].end) for start, end in sponsor_ranges]
        predicted_sponsor_times = []

        for times, text, label, prob in merge_predictions(predict_sponsor_segments(captions, window_duration), window_duration):
            if label == 'sponsor':
                predicted_sponsor_times.append((f'{int(prob * 100)}%', times))

            color = { 'sponsor': 'yellow', 'content': None }[label]

            if any((range_equals(times, actual_times, eps) for actual_times in sponsor_times)):
                close_matches += 1
            total_predicted_ranges += 1

        predicted_sponsor_ranges = [get_intersection_range(captions, *pair[1]) for pair in predicted_sponsor_times]
        predicted_labels = np.append(predicted_labels, create_labels_from_range(captions, predicted_sponsor_ranges))
        actual_labels = np.append(actual_labels, create_labels_from_range(captions, sponsor_ranges))
        print('p1',predicted_sponsor_ranges)
        try:

          predicted_sponsor_rangesii = predicted_sponsor_ranges                 #[item for item in predicted_sponsor_ranges if abs(item[1] - item[0]) >= 7]
          print('p2',predicted_sponsor_rangesii)
          predicted_sponsor_rangesiii=[]
          temp1=0
          if len(predicted_sponsor_rangesii) > 1:
            for i in predicted_sponsor_rangesii:
              temp=i[1]-i[0]
              if temp>temp1:
                temp1=temp
                predicted_sponsor_rangesiii.clear()
                predicted_sponsor_rangesiii.append(i)
              elif temp==temp1:
                #temp1=temp
                predicted_sponsor_rangesiii.clear()
                predicted_sponsor_rangesiii.append(i)
            predicted_sponsor_rangesiii = predicted_sponsor_rangesiii[0]

          elif len(predicted_sponsor_rangesii) == 1:
            predicted_sponsor_rangesiii = predicted_sponsor_rangesii[0]

          else:
            predicted_sponsor_rangesiii = (0,0)
        except:
            predicted_sponsor_rangesiii = (0,0)

        DistilBert = predicted_sponsor_rangesiii
        #print(f'\tPredicted = {predicted_sponsor_rangesiii}')

In [ ]:
def make_captions(vid_id):
  vid_id = vid_id                           #"YLt73w6criQ"
  output_path="/content/TestFolder1"
  url = f'https://www.youtube.com/watch?v={vid_id}'
  try:
      yt_dlp._real_main([
      "-4",
      "-o", output_path + "/%(id)s.%(ext)s",
      "--skip-download",
      "--write-sub",
      "--sub-lang", "en,en-AU,en-BZ,en-CA,en-IE,en-JM,en-NZ,en-ZA,en-TT,en-GB,en-US",
      url
      ])
  except Exception:
      DistilBert = None
      pass
  except SystemExit:
          DistilBert = None
          pass
  # Check for the output files (if any).
  files = glob(f'{output_path}/{vid_id}.*')
  if len(files) > 0:
      # Keep only the first result (in alphabetical order ~ *.en.*)
      for file in files[1:]:
          os.remove(file)
      filepath=['/content/TestFolder1']
      lenfolder = len([entry for entry in os.listdir(filepath[0]) if os.path.isfile(os.path.join(filepath[0], entry))])
      if lenfolder>0:
        main()
        test_videos2 = list(itertools.islice(load_captions_from_chunks2('data', '/content/TestFolder2', [1]), 0, 1))
        evaluate(videos=test_videos2,window_duration=10,eps=5,)
        !rm -r TestFolder1
        !rm -r TestFolder2
      else:
        DistilBert=None
  else:
      error = None
      DistilBert = None
      try:
        yt_dlp._real_main([ "-4", "-o", output_path + "/%(id)s.%(ext)s","--skip-download","--write-auto-sub",
        "--sub-lang", "en", url])
        # Check for the output files (if any).
        files = glob(f'{output_path}/{vid_id}.*')
        if len(files) > 0:
            # Keep only the first result (in alphabetical order ~ *.en.*)
            for file in files[1:]:
                os.remove(file)
            filepath=['/content/TestFolder1']
            lenfolder = len([entry for entry in os.listdir(filepath[0]) if os.path.isfile(os.path.join(filepath[0], entry))])
            if lenfolder>0:
              main()
              test_videos2 = list(itertools.islice(load_captions_from_chunks2('data', '/content/TestFolder2', [1]), 0, 1))
              evaluate(videos=test_videos2,window_duration=10,eps=5,)
              !rm -r TestFolder1
              !rm -r TestFolder2
            else:
              DistilBert=None
      except DownloadError:
          DistilBert = None
          error = f'Download error!'
      except SameFileError as e:
          error2 = f'There seems to exist a file with same name, Maybe check your files and delete the old .vtt file or rename it and try again!'
          error = str(e)
      except SystemExit:
          DistilBert = None
          pass
      files = glob(f'{output_path}/{vid_id}.*')

#LSTM

In [ ]:
def getContentFromURL(url):
   return requests.get(url).text

def vid_process(vid_id):
  global LSTM
  LSTM = 0
  trap2=0
  url = f'http://192.241.157.192/videos?part=mostReplayed&id={vid_id}'     #'https://yt.lemnoslife.com/videos?part=mostReplayed&id={vid_id}'

  content = getContentFromURL(url)
  data = json.loads(content)

  if(len(content) > 256):
        MV1=[]
        for heatMarker in data['items'][0]['mostReplayed']['heatMarkers']:
            heatMarker = heatMarker['heatMarkerRenderer']
            intensityScoreNormalized = heatMarker['heatMarkerIntensityScoreNormalized']
            a = heatMarker['timeRangeStartMillis']
            b = heatMarker['markerDurationMillis']
            MV1 += [intensityScoreNormalized]
            MV1 += [a]
            MV1 += [b]

        subList99 = [MV1[n:n+3] for n in range(0, len(MV1), 3)]
        subListtemp=[]
        subListtemp2=[]
        subListtemp3=[]
        subListtemp4=[]

        for i in range(len(subList99)):
            for j in range(len(subList99[i])):
              j=j-1
            subList99[i][j]=subList99[i][j]/1000   #range start ms to sec
            subListtemp.append(subList99[i][j])

        for i in range(len(subList99)):
            for j in range(len(subList99[i])):
              j=j
            subList99[i][j]=subList99[i][j]/1000   #marker duration ms to sec
            subListtemp3.append(subList99[i][j])

        for i in range(len(subList99)):
            for j in range(len(subList99[i])):    #rounding normalized score to 4 decimal places
              j=j-2
            subList99[i][j]=round(subList99[i][j],4)
            subListtemp2.append(subList99[i][j])

        for i in range(len(subList99)):
          if i==0:
            subListtemp4.append(vid_id)
          else:
            subListtemp4.append(np.NAN)
                                                  #[vid_id, heatMarkerIntensityScoreNormalized, IsSponsor , timeRangeStartMillis, markerDurationMillis]
        df99 = pd.DataFrame({'vid_id': pd.Series(pd.arrays.SparseArray(subListtemp4)),'Intensity_Score': subListtemp2, 'IsSponsor': '0',
                          'TimeRangeStart': subListtemp,
                          'Marker_Duration': subListtemp3})
        #print(df99)
        return df99  #[1,0.8,0.9] len 100
  else:
    LSTM = None
    return LSTM

In [ ]:
model_intrestgraph = keras.models.load_model('/content/TIMESERIES_LSTM.h5')

In [ ]:
# correct -FINALL- solution:
def find_dominant_value(values):
    # Create a histogram with 1000 bins
    hist, bin_edges = np.histogram(values, bins=1000)

    # Find the bin index with the highest frequency
    dominant_bin_index = np.argmax(hist)

    # Get the bin edges for the dominant bin
    dominant_bin_start = bin_edges[dominant_bin_index]
    dominant_bin_end = bin_edges[dominant_bin_index + 1]

    # Estimate the dominant value as the midpoint of the dominant bin
    dominant_value = (dominant_bin_start + dominant_bin_end) / 2

    # Round the dominant value to 6 decimal places
    rounded_dominant_value = round(dominant_value, 6)

    return rounded_dominant_value

def softmax(x):
    exlist=[]
    x_max = np.max(x)
    exlist = np.exp(x - x_max)
    #e_x = np.exp(x - 0.0000082561802864)
    return exlist / exlist.sum(axis=0) # only difference 0,998x

def calcv(ll):
  flagg1=0
  flagg2=0
  data1=0
  data2=0
  timestampstart=-1
  finaltimestampend=-1
  finaltimestampstart=0
  y=0
  startidx=0
  startidxb=0
  startidxb2=0
  datal=[]
  list3=[]
  temp1=0
  ididit=0
  dupfinaltimestampend2ii=0
  finaltimestampend2=0
  dupfinaltimestampend2=0
  subt1=0
  tempn,tempn1,tempn2=0,0,0
  prevflagg1=0
  for i in range(len(ll)):
    if ll[i]==1:
      flagg1=flagg1+1
      y+=1
      if timestampstart==-1:
        timestampstart=i
      flagg2=0
    else:
      flagg2=flagg2+1
      if flagg2<=2 and i>6 and y<=2: y+=1
      if flagg2>2:
        startidx+=1
        flagg1=0
        timestampstart=-1
        flagg2=0
        y=0
        if startidx == 1:
          startidxb2=i-2
          startidxb=9999 #starting all high vals done

    if (startidxb==9999) or (startidxb==99999):
      if y>2 :
        x=finaltimestampstart
        finaltimestampstart=timestampstart
        finaltimestampend=i
        #print(finaltimestampstart,x)
        #print('i',i)
        if x!=finaltimestampstart and finaltimestampend!=finaltimestampend2 and x!=finaltimestampend and x!=0 and finaltimestampend2!=startidxb2 and ((finaltimestampend-x)>6) or i==len(ll)-1:
          #print(ididit)
          if ididit==0:
            if startidxb==9999 and i>6:
              datal.clear()
              startidxb=99999
            else:
              #datal.clear()
              startidxb=99999

            if i==len(ll)-1:
              finaltimerange=(prevflagg1,x,finaltimestampend2+1)
              datal.append(finaltimerange)
            else:
              finaltimerange=(prevflagg1,x,finaltimestampend2)
              datal.append(finaltimerange)
        prevflagg1=flagg1
        finaltimestampend2=i

    else:
      if y>2 :
        x=finaltimestampstart
        finaltimestampstart=timestampstart
        finaltimestampend=i

        if x!=finaltimestampstart or finaltimestampend!=finaltimestampend2 and x!=finaltimestampend  and ((finaltimestampend-x)>=4) or i>2:
          finaltimerange=(flagg1,x,finaltimestampend2)
          datal.clear()
          datal.append(finaltimerange)
          dupfinaltimestampend2=i
        finaltimestampend2=i
        if dupfinaltimestampend2ii==1:
          subt1=1
          dupfinaltimestampend2ii=99999
        elif len(datal)>0 and subt1==1:
          subt1=0
        if i > (dupfinaltimestampend2-x-subt1)  or dupfinaltimestampend2>6 or flagg1>6:

            ididit=0
        else:
          for k in range(i,11):
            if ll[k] == 1:
              ididit=1
            elif ll[k] == 0:
              ididit=0

  for i in datal:
    temp=i[0]
    if temp>temp1:
      temp1=temp
      list3.clear()
      list3.append((i[1],i[2]))
    if len(datal)==0 or (i[1]==0 and i[2]-i[1]>6):
        list3.clear()
        list3.append((0,0))

  return datal



In [ ]:
def final_lstm_prediction(yt_test_timeseries):
  global LSTM, model_intrestgraph
  flag1 = 0
  if LSTM != None:
    testt=[]
    jj=0
    rawarray=yt_test_timeseries[['Intensity_Score']]
    for i in range(0,rawarray.shape[0],100):
      testt.append(np.reshape(np.array(rawarray[i:i+100]),(jj+1, 100,rawarray.shape[1])))
    test1i = np.array(testt)
    #print('tes',test1i)
    xttii = np.reshape(np.array(test1i),(test1i.shape[0], 100,1))
    prediii = model_intrestgraph.predict(xttii)

    z1=yt_test_timeseries['TimeRangeStart']
    z1dup=int(z1[1]-z1[0])
    #print(z1dup)
    #iparray = prediii[0,:]
    maxval = 0
    coeff = 0 #40%
    flag2 = 0
    datatimestart=-1
    datatimestop=0

    asq = np.array(prediii.reshape(100))
    sof1 = softmax(asq)
    sof1=sof1*100
    sof1 = np.round_(sof1,6)
    q=find_dominant_value(sof1)

    value = q

    last_three_digits = int(value * 1000000) % 1000

    if last_three_digits // 100 % 10 == 0:
        output_threedigit = f"0.00{last_three_digits:03d}"
    else:
        output_threedigit = f"0.000{last_three_digits:03d}"

    output_threedigit = float(output_threedigit)
    #print(output_threedigit)

    # Convert the number to a string
    number_str = str(value)

    # Extract the first three numbers
    first_three_numbers = number_str[2:7]
    first_three_numbers = int(first_three_numbers)


    ll=[]
    for i in range(1,(len(sof1))):
      if abs(sof1[i]-sof1[i-1]) > output_threedigit:
        ll.append(1)
      else:
        ll.append(0)

    if first_three_numbers < 99830:  #NON_SPONS
      LSTM = (0,0)
      #print(LSTM)
    else:                            #SPONS
      tq = calcv(ll)
      #print(tq)
      try:
        start_tq = int(z1[tq[0][1]])
        end_tq = int((z1[tq[0][2]]))
      except:
        start_tq = 0
        end_tq = 0
      #print(z1[tq[0]])
      tq2 = (start_tq,end_tq)
      LSTM = tq2

  else:
    LSTM = None
  return LSTM

#CNN

In [ ]:
# now, for the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_parent = torch.hub.load('facebookresearch/deit:main', 'deit_small_patch16_224', pretrained=True)
for param in model_parent.parameters(): #freeze model
    param.requires_grad = False

n_inputs = model_parent.head.in_features
model_parent.head = nn.Sequential(
    nn.Linear(n_inputs, 2048),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(2048, 1524),
    nn.ReLU(),
    nn.Linear(1524, 1024),
    nn.Dropout(0.25),
    nn.Linear(1024, 800),
    nn.Linear(800, 512),
    nn.Linear(512, 2)  #2 classes ----/
)
model2 = model_parent.to(device)

# Define the dropout layer to be added
dropout_layer = nn.Dropout(0.3)

# Get the reference to the existing layer
old_conv = model2.head[7]

# Create a new sequential module with the dropout layer followed by the existing layer
new_conv = nn.Sequential(
    old_conv, dropout_layer
)
# Update the model with the new sequential module
model2.head[7] = new_conv

print(model2.head)

checkpoint = torch.load("/content/DeiT_CNN.pt", map_location=torch.device('cpu'))
optimizer = torch.optim.AdamW(model2.head.parameters(), lr=0.001, weight_decay=0.1)
model2.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
criterion = LabelSmoothingCrossEntropy()
criterion = criterion.to(device)
optimizer.param_groups[0]['weight_decay'] = 0.1

Downloading: "https://github.com/facebookresearch/deit/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/deit/deit_small_patch16_224-cd65a155.pth" to /root/.cache/torch/hub/checkpoints/deit_small_patch16_224-cd65a155.pth
100%|██████████| 84.2M/84.2M [00:00<00:00, 160MB/s]


Sequential(
  (0): Linear(in_features=384, out_features=2048, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=2048, out_features=1524, bias=True)
  (4): ReLU()
  (5): Linear(in_features=1524, out_features=1024, bias=True)
  (6): Dropout(p=0.25, inplace=False)
  (7): Sequential(
    (0): Linear(in_features=1024, out_features=800, bias=True)
    (1): Dropout(p=0.3, inplace=False)
  )
  (8): Linear(in_features=800, out_features=512, bias=True)
  (9): Linear(in_features=512, out_features=2, bias=True)
)


In [ ]:
def predict_fps(vid_id1):
  global model2
  transform = T.Compose([ # We dont need augmentation for test transforms
              T.Resize(256),
              T.CenterCrop(224),
              T.ToTensor(),
              T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
          ])
  filepath=["/content/TestFolderCNN/"]
  list1=[]
  fw = 1
  ll=[]
  !rm -r TestFolder_I/.ipynb_checkpoints > /dev/null 2>&1
  lenfolder = len([entry for entry in os.listdir(filepath[0]) if os.path.isfile(os.path.join(filepath[0], entry))])
  while fw <= lenfolder:
      fw2 = vid_id1+'_'+str(fw)+'.jpg'
      temp=str(filepath[0])+fw2
      #print(temp)
      image1=Image.open(temp)
      image1=transform(image1)
      image1=image1.view(1,3,224,224)
      image1=image1.cuda()             #Removee cuda() if using cpu
      model2.eval()
      output=model2(image1)
      prediction=int(torch.max(output.cpu().data,1)[1].numpy())    #torch.softmax(output, dim=1)
      list1.append(prediction)
      fw+=1
  !rm -r TestFolderCNN
  return list1



def preprocess_frames(vid_id):
  folder_path = './TestFolderCNN'
  os.makedirs(folder_path, exist_ok=True)
  os.listdir('.')
  vid_id1=vid_id
  str_command1 = f''' ! ffmpeg -i $(yt-dlp -f worstvideo --get-url "https://www.youtube.com/watch?v={vid_id1}") -vf fps=1 /content/TestFolderCNN/{vid_id1}_%d.jpg > /dev/null 2>&1 '''
  os.system(str_command1)
  list1 = predict_fps(vid_id1)
  return list1

In [ ]:
def DeLT_Calculate(vid_id):
  make_captions(vid_id)

  yt_test_timeseries=vid_process(vid_id)
  final_lstm_prediction(yt_test_timeseries)

  preprocess_framesii = preprocess_frames(vid_id)
  calculate_timerangeDeiT(preprocess_framesii)

In [ ]:
DeLT_Calculate('RFfIjOOJbcM')
print(DistilBert)
print(LSTM)
#print(DeiT) #340-420

[youtube] Extracting URL: https://www.youtube.com/watch?v=RFfIjOOJbcM
[youtube] RFfIjOOJbcM: Downloading webpage
[youtube] RFfIjOOJbcM: Downloading android player API JSON
[info] RFfIjOOJbcM: Downloading subtitles: en
[info] RFfIjOOJbcM: Downloading 1 format(s): 248+251
[info] Writing video subtitles to: /content/TestFolder1/RFfIjOOJbcM.en.vtt
[download] Destination: /content/TestFolder1/RFfIjOOJbcM.en.vtt
[download] 100% of   11.93KiB in 00:00:00 at 386.53KiB/s
Processing captions...
100.00% elapsed: 0:00:00, remaining: 0:00:00

Opening /content/TestFolder2/data.1.json.gz for reading...
Closed /content/TestFolder2/data.1.json.gz.
  0%|          | 0/1 [00:00<?, ?it/s]

Input Video ID: RFfIjOOJbcM


100%|██████████| 1/1 [00:00<00:00,  1.41it/s]

p1 [(68, 73), (None, None)]


(0, 0)
(0, 0)


In [ ]:
DeLT_Calculate('7Qi6oIOHbDg')
print(DistilBert)
print(LSTM)
print(DeiT) #247-300

1/1 [==============================] - 0s 48ms/step
[(2, 95, 98)]
0
(286, 295)
0


In [ ]:
DeLT_Calculate('5CUBJgknoW0')  #no spons no time g
print(DistilBert)
print(LSTM)
print(DeiT)

1/1 [==============================] - 0s 69ms/step
0
(0, 0)
0


In [ ]:
DeLT_Calculate('p1F0bMlGkXU')  #no spons no time g no sub
print(DistilBert)
print(LSTM)
print(DeiT)

[youtube] Extracting URL: https://www.youtube.com/watch?v=p1F0bMlGkXU
[youtube] p1F0bMlGkXU: Downloading webpage
[youtube] p1F0bMlGkXU: Downloading android player API JSON
[info] p1F0bMlGkXU: Downloading 1 format(s): 303+251
[info] There's no subtitles for the requested languages
[youtube] Extracting URL: https://www.youtube.com/watch?v=p1F0bMlGkXU
[youtube] p1F0bMlGkXU: Downloading webpage
[youtube] p1F0bMlGkXU: Downloading android player API JSON
[info] p1F0bMlGkXU: Downloading subtitles: en
[info] p1F0bMlGkXU: Downloading 1 format(s): 303+251
[info] Writing video subtitles to: /content/TestFolder1/p1F0bMlGkXU.en.vtt
[download] Destination: /content/TestFolder1/p1F0bMlGkXU.en.vtt
[download] 100% of   14.60KiB in 00:00:00 at 107.49KiB/s
Processing captions...
100.00% elapsed: 0:00:00, remaining: 0:00:00

Opening /content/TestFolder2/data.1.json.gz for reading...
Closed /content/TestFolder2/data.1.json.gz.
  0%|          | 0/1 [00:00<?, ?it/s]

Input Video ID: p1F0bMlGkXU


100%|██████████| 1/1 [00:28<00:00, 28.70s/it]

p1 [(25, 27)]
p2 []
	Predicted = (0, 0)


1/1 [==============================] - 0s 63ms/step
(0, 0)
(0, 0)
(436, 457)


In [ ]:
DeLT_Calculate('E5HRvQNg4pQ')  #263-320
print(DistilBert)
print(LSTM)
print(DeiT)

1/1 [==============================] - 0s 64ms/step
[(10, 16, 27), (14, 38, 53), (3, 55, 60), (3, 95, 98)]
0
(54, 92)
0


In [ ]:
DeLT_Calculate('IoTL9xZOdP0')
print(DistilBert)
print(LSTM)
print(DeiT)

[youtube] Extracting URL: https://www.youtube.com/watch?v=IoTL9xZOdP0
[youtube] IoTL9xZOdP0: Downloading webpage
[youtube] IoTL9xZOdP0: Downloading android player API JSON
[info] IoTL9xZOdP0: Downloading 1 format(s): 248+251
[info] There's no subtitles for the requested languages
[youtube] Extracting URL: https://www.youtube.com/watch?v=IoTL9xZOdP0
[youtube] IoTL9xZOdP0: Downloading webpage
[youtube] IoTL9xZOdP0: Downloading android player API JSON
[info] IoTL9xZOdP0: Downloading 1 format(s): 248+251
[info] There's no subtitles for the requested languages
1/1 [==============================] - 0s 82ms/step
(0, 0)
[(4, 43, 49), (1, 72, 74), (1, 92, 95)]
(531, 545)


#FINAL WEIGHTS

In [ ]:
def finalmerge(in1,in2,in3):  #flow chart, avg duration
  if in1!=None and in2!=None and in3!=None:
    try:
      final=(int(in1[0]*0.8+in2[0]*0.05+in3[0]*0.15),int(in1[1]*0.8+in2[1]*0.05+in3[1]*0.15))
    except:
      final=(int(in3[0]*0.85+in2[0]*0.15),int(in3[1]*0.85+in2[1]*0.15))
    # final=list(final)
    if (in2[1]==0 and in3[1]==0):
       final=(in1[0],in1[1])
    if (in1[0]==0 and in1[1]==0):
      final=(0,0)
    if (final[1]-final[0]<=0 and final[1]!=None or final[0]!=None):
      final=(0,0)
    else:
      final=(0,0)
    print("ALL Available")
  elif in1!=None and in2==None and in3!=None:
    try:
      final=(int(in1[0]*0.65+in3[0]*0.35),int(in1[1]*0.65+in3[1]*0.35))
    except:
       final=(in3[0],in3[1])
    #if (in1[0]==0 and in1[1]==0):
    #  final=(0,0)
    print("NLP & CNN Available")
  elif in2!=None and in1==None and in3!=None:
    final=(int(in2[0]*0.1+in3[0]*0.9),int(in2[1]*0.1+in3[1]*0.9))
    if (in2[0]==0 and in2[1]==0):
      final=(in3[0],in3[1])
    print("LSTM & CNN Available")
  elif in2==None and in1==None and in3!=None:
    final=(in3[0],in3[1])
    print("Only CNN Available")
  else:
      print('oopsieee:',type(in1),type(in2),type(in3))
      final=(0,0)

  return final

In [ ]:
from termcolor import colored
IOU=[]
accu=[]
pred_labels = np.empty(0)
actual_labels = np.empty(0)
def create_iou(start_gt,end_gt,len_gt,start_pred,end_pred,len_pred,length_in_seconds):
    global IOU,accu,pred_labels,actual_labels

    # Step 1: Convert the range format to individual values

    prediction_values = list(range(start_pred, end_pred + 1))
    ground_truth_values = list(range(start_gt, end_gt + 1))

    overlap_threshold = 0.4  # Adjust this threshold based on your problem
    num_matches = sum(1 for val in prediction_values if val in ground_truth_values)
    accuracy = num_matches / len(ground_truth_values)
    accu.append(accuracy)
    bigtemp=length_in_seconds
    acc_gt_labels = np.zeros(bigtemp, dtype=bool)
    acc_pred_labels = np.zeros(bigtemp, dtype=bool)
    for i in range(start_gt,end_gt + 1):
      try:
        acc_gt_labels[i] = True
      except:
        pass
    for i in range(start_pred,end_pred + 1):
      try:
        acc_pred_labels[i] = True
      except:
        pass

    actual_labels = np.append(actual_labels,acc_gt_labels)
    pred_labels = np.append(pred_labels,acc_pred_labels)

    #numerator = max(0,min(end_pred,end_gt))
    #denominator = len_gt+len_pred-numerator

    intersection_lower = max(start_pred, start_gt)
    intersection_upper = min(end_pred, end_gt)

    intersection_width = intersection_upper - intersection_lower + 1 #(smoothing)  Predicted=(95, 110),Expected=[[0, 2]]
    if intersection_width<0:
      intersection_width=0
    union_width = max(end_pred, end_gt) - min(start_pred, start_gt) + 1
    final_iou = intersection_width / union_width
    #final_iou = numerator/denominator
    IOU.append(final_iou)
def get_video_length(url):
  try:
    with yt_dlp.YoutubeDL({}) as ydl:
        info = ydl.extract_info(url, download=False)
        duration = info.get('duration', 0)
        return int(duration)
  except:
    return 1000


def evaluate_FINALE(videos, eps=5, window_duration=10):
    from tqdm.auto import tqdm
    global IOU,actual_labels,pred_labels
    IOU.clear()
    #predicted_labels = np.empty(0)
    #actual_labels = np.empty(0)

    # Values for our close match metric (exact match with threshold)
    # Number of matches

    # Number of predicted ranges
    total_predicted_ranges = 0
    iterate=0

    for video_id,captions,sponsor_ranges in tqdm(videos):
        length_in_seconds = get_video_length(video_id)
        if length_in_seconds < 901:
          iterate+=1
          print(colored(f'{video_id} {sponsor_ranges}', None, 'on_magenta'))
          #sponsor_times = [(captions[start].start, captions[end].end) for start, end in sponsor_ranges]
          predicted_sponsor_times = []

          DeLT_Calculate(video_id)
          iter = 0
          temp1=0
          predicted_sponsor_ranges = finalmerge(DistilBert,LSTM,DeiT)
          if len(sponsor_ranges) > 1:
            for i in sponsor_ranges:
              temp=i[1]-i[0]
              if temp>temp1:
                temp1=temp
                sponsor_ranges.clear()
                sponsor_ranges.append(i)
              elif temp == temp1 and iter>0:
                temp1=temp
                sponsor_ranges.clear()
                sponsor_ranges.append(i)
              iter +=1

            print('After reshape: ',sponsor_ranges)
          if len(sponsor_ranges) > 0:
            sponsor_rangesii = np.reshape(sponsor_ranges, (2))
          else:
            sponsor_ranges = [[0,0]]
            sponsor_rangesii=(0,0)
          start_gt = sponsor_rangesii[0]
          start_pred = predicted_sponsor_ranges[0]
          end_gt = sponsor_rangesii[1]
          end_pred = predicted_sponsor_ranges[1]
          len_pred = end_pred - start_pred
          len_gt = end_gt-start_gt
          #print(start_gt, start_pred, end_gt, end_pred, len_pred, len_gt)  #0 95 2 110 15 2    53 241 73 246 5 20
          start_gt,end_gt,len_gt,start_pred,end_pred,len_pred = int(start_gt),int(end_gt),int(len_gt),int(start_pred),int(end_pred),int(len_pred)
          create_iou(start_gt,end_gt,len_gt,start_pred,end_pred,len_pred,length_in_seconds)
          print('NLP:',DistilBert)
          print('LSTM:',LSTM)
          print('DeiT',DeiT)
          #predicted_sponsor_ranges = [get_intersection_range(captions, *pair[1]) for pair in predicted_sponsor_times]
          #predicted_labels = np.append(predicted_labels, create_labels_from_range(captions, predicted_sponsor_ranges))
          #actual_labels = np.append(actual_labels, create_labels_from_range(captions, sponsor_ranges))

          print(f'\tPredicted={predicted_sponsor_ranges},\n\tExpected={sponsor_ranges}')

          if iterate % 10 == 0:
            npIOU = np.array(IOU)
            newIOU=np.mean(npIOU)
            print('IOU:',newIOU)

            print('Confusion matrix', confusion_matrix(actual_labels, pred_labels))
            print('Accuracy', accuracy_score(actual_labels, pred_labels))
            print('Precision', precision_score(actual_labels, pred_labels))
            print('Recall', recall_score(actual_labels, pred_labels))
        else:
          pass

    npIOU = np.array(IOU)
    newIOU=np.mean(npIOU)
    print('IOU:',newIOU)
    print('Accuracy:',accu)
    print('Confusion matrix', confusion_matrix(actual_labels, pred_labels))
    print('Accuracy', accuracy_score(actual_labels, pred_labels))
    print('Precision', precision_score(actual_labels, pred_labels))
    print('Recall', recall_score(actual_labels, pred_labels))

In [ ]:
test_videos = list(itertools.islice(load_data_from_chunks('data', '/content/drive/MyDrive/JSON', [16]), 1, 501))

Opening /content/drive/MyDrive/JSON/data.16.json.gz for reading...


In [ ]:
len(test_videos)

500

In [ ]:
evaluate_FINALE(test_videos)  #latest 1/6/23 8:15 PM  #highest IOU: 65.6%

  0%|          | 0/500 [00:00<?, ?it/s]

[youtube] Extracting URL: 79rP8OJ-Wao
[youtube] 79rP8OJ-Wao: Downloading webpage
[youtube] 79rP8OJ-Wao: Downloading android player API JSON
[youtube] Extracting URL: L0TM5FWgBWk
[youtube] L0TM5FWgBWk: Downloading webpage
[youtube] L0TM5FWgBWk: Downloading android player API JSON
L0TM5FWgBWk []
[youtube] Extracting URL: https://www.youtube.com/watch?v=L0TM5FWgBWk
[youtube] L0TM5FWgBWk: Downloading webpage
[youtube] L0TM5FWgBWk: Downloading android player API JSON
[info] L0TM5FWgBWk: Downloading 1 format(s): 313+251
[info] There's no subtitles for the requested languages
[youtube] Extracting URL: https://www.youtube.com/watch?v=L0TM5FWgBWk
[youtube] L0TM5FWgBWk: Downloading webpage
[youtube] L0TM5FWgBWk: Downloading android player API JSON
[info] L0TM5FWgBWk: Downloading subtitles: en
[info] L0TM5FWgBWk: Downloading 1 format(s): 313+251
[info] Writing video subtitles to: /content/TestFolder1/L0TM5FWgBWk.en.vtt
[download] Destination: /content/TestFolder1/L0TM5FWgBWk.en.vtt
[download] 100

Opening /content/TestFolder2/data.1.json.gz for reading...
Closed /content/TestFolder2/data.1.json.gz.

  0%|          | 0/1 [00:00<?, ?it/s]

Input Video ID: L0TM5FWgBWk



100%|██████████| 1/1 [00:01<00:00,  1.26s/it]

...

Opening /content/TestFolder2/data.1.json.gz for reading...
Closed /content/TestFolder2/data.1.json.gz.

  0%|          | 0/1 [00:00<?, ?it/s]

Input Video ID: 9TCkdvcfDg0



100%|██████████| 1/1 [00:00<00:00,  1.78it/s]

p1 [(0, 3)]
p2 [(0, 3)]


1/1 [==============================] - 0s 32ms/step
ALL Available
NLP: (0, 3)
LSTM: (0, 0)
DeiT (85, 99)
	Predicted=(0, 0),
	Expected=[[0, 0]]
IOU: 0.6566671176046177
Accuracy: [0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.09090909090909091, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.16666666666666666, 0.0, 0.3333333333333333, 0.0, 0.15384615384615385, 0.0, 1.0, 0.2, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.027777777777777776, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.